In [1]:
import numpy as np, random, operator, pandas as pd

In [2]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance

    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [3]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0

    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

In [4]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [5]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

In [6]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)


In [7]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()

    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [8]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [9]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []

    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])

    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

In [10]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])

    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [11]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))

            city1 = individual[swapped]
            city2 = individual[swapWith]

            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [12]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []

    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [13]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

In [14]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    progress = []
    progress.append(1 / rankRoutes(pop)[0][1])

    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])

    return progress


In [15]:
%%time
cityList = []

for i in range(0,5):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#maxiter indicates then number of iterations per run
maxiter = 100

numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popzise=population size,elite 20% of total population
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter)

    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Promedios de las iteraciones")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Promedios de las iteraciones
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870428123
361.870

In [16]:
%%time
cityList = []

for i in range(0,50):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#maxiter indicates then number of iterations per run
maxiter = 100

numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popzise=population size,elite 20% of total population
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter)

    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Promedios de las iteraciones")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Promedios de las iteraciones
4431.552172526
4329.317804158
4235.854038085
4150.808111983
4076.157713155
4013.637388840
3947.471745470
3890.874516696
3835.329582358
3787.928712172
3728.509736402
3684.098617092
3646.890969268
3614.907938725
3587.837276467
3545.176588936
3511.036533307
3482.479869527
3449.561324082
3418.313364956
3394.896859837
3362.310054384
3348.701616155
3325.254138972
3292.893323146
3284.692274972
3262.974215887
3239.919128039
3225.770513314
3212.667259136
3190.685997114
3170.911963904
3150.330713864
3138.707264141
3110.599162049
3105.923282222
3091.168365106
3073.961080196
3051.210180104
3034.494088038
3026.490380959
3013.671047347
3010.827998155
2991.577798790
2978.528053251
2962

In [17]:
%%time
cityList = []

for i in range(0,500):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#maxiter indicates then number of iterations per run
maxiter = 100

numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popzise=population size,elite 20% of total population
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter)

    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Promedios de las iteraciones")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Promedios de las iteraciones
50169.298098616
49935.428436360
49682.138370174
49426.521725823
49164.863856380
48947.226321122
48751.515978911
48494.694565778
48349.266144313
48211.249974014
48030.634591028
47930.898601790
47810.403878580
47629.009563966
47564.760664030
47444.883870592
47373.946901487
47320.624877897
47175.748519942
47091.729303847
47021.027394253
46917.428837311
46871.593863981
46847.151266460
46777.179490179
46688.252459948
46610.872159615
46553.883833483
46504.324645597
46468.911800291
46438.528011477
46392.180393909
46380.654245821
46319.147719079
46289.493767914
46262.528476988
46241.734708649
46185.860482735
46223.222370171
46201.816253897
46185.618631884
46200.844114023
46118.8

In [15]:
%%time
cityList = []

for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#maxiter indicates then number of iterations per run
maxiter = 100

numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popzise=population size,elite 20% of total population
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter)

    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Promedios de las iteraciones")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Promedios de las iteraciones
1849.183849716
1786.321491609
1733.975733271
1688.959779730
1657.986041433
1617.054412969
1588.876664214
1556.382063300
1529.637185204
1504.367398212
1476.730274720
1454.898573239
1436.962409924
1419.947071276
1408.414840024
1386.675482646
1374.056451507
1359.289818503
1345.798099938
1331.411218764
1313.142038431
1298.234091035
1290.865212185
1280.504722449
1271.887328653
1265.050083993
1256.332577188
1248.144227334
1240.960200956
1235.022385239
1227.591379628
1218.331211493
1209.596722385
1205.427597043
1198.240561093
1188.783993302
1184.862010918
1176.857148538
1175.210159477
1169.495424382
1165.280357026
1158.968150821
1153.443449254
1148.657791187
1140.191748956
1137

In [16]:
%%time
cityList = []

for i in range(0,1000):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []
promedios = []
#maxiter indicates then number of iterations per run
maxiter = 100

numero_iteraciones = 100

for i in range(0,numero_iteraciones):
    print(i)
    #popzise=population size,elite 20% of total population
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter)

    iteraciones.append(iteracionTemporal)

for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]

    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)

print("\n")
print("Promedios de las iteraciones")
for i in range(0,maxiter):
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Promedios de las iteraciones
101954.068204919
101464.529876929
100969.717325990
100591.639573417
100280.916733155
99942.848450676
99723.027734418
99514.282942065
99254.583669773
98981.093275588
98756.290415893
98545.482274586
98424.124035113
98299.465534147
98093.664184435
98026.482871970
97857.040135484
97781.259519690
97616.466559693
97493.584896867
97437.638129834
97301.437839802
97196.635336127
97122.771994080
96981.160962242
96920.408253916
96811.066517488
96755.125864449
96737.781453403
96729.607730723
96676.442787643
96554.198363414
96512.031124608
96463.511327932
96473.553598156
96445.651004627
96394.001498314
96364.454618724
96251.184681832
96178.088162105
96215.521879196
96146.165193703
96